# Cloud Computing Assignment 2022-2023
Implementation of an application processing large data sets in parallel on a distributed Cloud environment (ie. AWS)

© Copyright 2022, All rights reserved to Hans Haller, CSTE-CIDA Student at Cranfield Uni. SATM, Cranfield, UK.

### Solution setup - Pre-requisites:
1. Make sure the aws credentials taken from the Learner Lab are updated in the ~/.aws/credentials file (Test connection locally using aws sts get-caller-identity)
2. Specify the "labsuser.pem" perm-key's (taken from the Learner Lab) path, needed by paramiko to connect to the EC2 instances and execute ssh commands.
3. Create EC2, S3 and SQS resources and clients using boto3.
### Solution setup steps (Using Boto3):
1. Create a cluster of EC2 instances on AWS, using the AWS Linux 2 images.
2. Create a S3 bucket to store the data.
3. Create a SQS queue to store stacks of messages.

### IMPORTS:

The following controllers defines functions that use boto3 packaged functions to make AWS API calls. By importing the controllers, a Boto3 resource is automatically created for each element of the solution (EC2, SQS, SSM, S3, etc) in order for these functions to work. The Boto3 resources uses the AWS credentials that are located in the .aws local folder of the user who executes this software. As a result, it is important that they are updated before running the following. Thus please make sure to restart the kernel and re-execute the imports if the credentials expired (ie. the Learner Lab session ended).

In [ ]:
from backend.controllers.boto3_controller import *
from backend.controllers.ec2_controller import *
from backend.controllers.matrix_controller import *
from backend.controllers.s3_controller import *
from backend.controllers.spark_controller import *
from backend.controllers.sqs_controller import *
from backend.controllers.ssm_controller import *
from backend.controllers.app_controller import *

In [ ]:
from backend.work_service import *

In [ ]:
# TODO: Replace every pre-steps of exec_shell() with the following procedure:
# Create a setup.sh script that installs all the packages needed for the application to run, and then execute it on each instance.

# TODO (BONUS): BULK VERIFICATIONS

## AWS - SOLUTION SETUP AND TASKS EXECUTION:

In [ ]:
# SETTINGS=
worker_amount = 2
matrix_shape = 1000
backend_path = os.path.join(os.getcwd(), 'backend')     # !! IMPORTANT: Make sure to update this path to the backend folder of the project !!

# NAMES=
instances_names = np.concatenate((np.array(['master']), np.array(['worker' + str(i) for i in range(1, worker_amount+1)]))).tolist()
queues_names = ['main-protected-jobs.fifo', 'main-protected-results.fifo']
bucket_name = 'main-protected-bucket'

In [ ]:
# EC2 INSTANCES:
create_instances_and_wait_for_running(instances_names)
# SQS QUEUES:
create_sqs_queues(queues_names)
# S3 BUCKET:
create_s3_bucket(bucket_name)

In [ ]:
print(get_instance_public_dns_by_name(instances_names[0]))

### ENVIRONMENT SETUP:

In [ ]:
# S3 & EC2 - BACKEND FOLDER SETUP:
# UPLOAD BACKEND FOLDER -> S3 BUCKET:
upload_dir_to_s3(backend_path, bucket_name, 'backend')
for instance_name in instances_names:
    # DOWNLOAD BACKEND FOLDER : S3 BUCKET -> EC2 INSTANCES:
    download_directory_on_instance_from_s3_bucket(instance_name, bucket_name, 'backend', 'backend')
    # CREATE THE TEMP DATA/INPUT AND DATA/OUTPUT DIRECTORIES:
    exec_SSH_on_instance(instance_name, 'cd backend && mkdir data && cd data && mkdir input && mkdir output && cd output && mkdir mx')

In [ ]:
# EC2 INSTANCES - PACKAGE DOWNLOAD:
for instance_name in instances_names:
    update_instance_credentials_using_boto3_session_credentials(instance_name)
    exec_SSH_on_instance(instance_name, 'sudo yum install tree -y')
    exec_SSH_on_instance(instance_name, 'pip3 install boto3')
    exec_SSH_on_instance(instance_name, 'pip3 install numpy')
    exec_SSH_on_instance(instance_name, 'pip3 install pandas')

In [ ]:
# create_matrix_then_split_and_send_jobs(matrix_shape, queues_names[0], bucket_name)
# gather_jobs_then_compute_and_send_results(queues_names[0], queues_names[1])
# gather_results_and_reconstruct_matrix(queues_names[1], bucket_name)
# verify_result_matrix(bucket_name, '707566791', 'mx')

### SOLUTION EXECUTION:

In [ ]:
start_time = time.time()

In [ ]:
# JOB: CREATE A MATRIX, SPLIT IT INTO SLICES, AND SEND THE SLICES TO THE JOBS QUEUE:
command = 'python3 backend/work_service.py create ' + str(matrix_shape) + ' ' + queues_names[0] + ' ' + bucket_name
stdout, stderr = exec_SSH_on_instance(instances_names[0], command)
outprint(stdout, stderr)

In [ ]:
# JOB: GATHER JOBS FROM THE JOBS QUEUE, COMPUTE THE RESULTS, AND SEND THE RESULTS TO THE RESULTS QUEUE:
for instance_name in instances_names[1:2]:
    command = 'python3 backend/work_service.py getjobs ' + queues_names[0] + ' ' + queues_names[1]
    stdout, stderr = exec_SSH_on_instance(instance_name, command)
    outprint(stdout, stderr)

In [ ]:
# JOB: GATHER RESULTS FROM THE RESULTS QUEUE, AND RECONSTRUCT THE RESULT MATRIX:
command = 'python3 backend/work_service.py getresults ' + queues_names[1] + ' ' + bucket_name
stdout, stderr = exec_SSH_on_instance(instances_names[0], command)
outprint(stdout, stderr)

In [ ]:
print('Computation time: ' + str(time.time() - start_time) + ' seconds.')

In [ ]:
start_time = time.time()
# JOB: CREATE A MATRIX, SPLIT IT INTO SLICES, AND SEND THE SLICES TO THE JOBS QUEUE:
command = 'python3 backend/work_service.py create ' + str(matrix_shape) + ' ' + queues_names[0] + ' ' + bucket_name
stdout, stderr = exec_SSH_on_instance(instances_names[0], command)
# JOB: GATHER JOBS FROM THE JOBS QUEUE, COMPUTE THE RESULTS, AND SEND THE RESULTS TO THE RESULTS QUEUE:
for instance_name in instances_names[1:]:
    command = 'python3 backend/work_service.py getjobs ' + queues_names[0] + ' ' + queues_names[1]
    exec_SSH_on_instance_no_wait(instance_name, command)
# JOB: GATHER RESULTS FROM THE RESULTS QUEUE, AND RECONSTRUCT THE RESULT MATRIX:
command = 'python3 backend/work_service.py getresults ' + queues_names[1] + ' ' + bucket_name
exec_SSH_on_instance(instances_names[0], command)
print('Computation time: ' + str(np.round(time.time() - start_time, 2)) + ' seconds.')

# To fix, make infinite loop in the getjobs function, and add a break condition when the process received as much blocks as it should have received.
# The job amount can be known at any time as it is in the body of every SQS message.

In [ ]:
id = '117988589'
op = 'mx'

In [ ]:
# JOB: VERIFY THE RESULTS COMPUTED ON AWS DISTRIBUTED CLOUD ENVIRONMENT, USING NUMPY'S MATRIX/ADD FUNCTIONS:
command = 'python3 backend/work_service.py verify ' + bucket_name + ' ' + id + ' ' + op
stdout, stderr = exec_SSH_on_instance(instances_names[0], command)
outprint(stdout, stderr)

### CLEAN UP:

In [ ]:
for queue_name in queues_names:
    purge_queue(queue_name)
    delete_sqs_queue(queue_name)
for instances_name in instances_names:
    stop_instance_by_name(instances_name)
    terminate_instance_by_name(instances_name)
delete_s3_bucket(bucket_name)